# **Train**

In [1]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 11.4 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/wilwil2468/IRSRCNN.git

Cloning into 'IRSRCNN'...
remote: Enumerating objects: 12611, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 12611 (delta 47), reused 56 (delta 32), pack-reused 12539 (from 2)
Receiving objects: 100% (12611/12611), 1.12 GiB | 40.43 MiB/s, done.
Resolving deltas: 100% (259/259), done.
Updating files: 100% (12014/12014), done.


In [3]:
%cd /kaggle/working/IRSRCNN

/kaggle/working/IRSRCNN


In [4]:
!git pull

Already up to date.


For “915” (k9–k1–k5) you used hr_crop_size = 21

For “935” (k9–k3–k5) you used hr_crop_size = 19

For “955” (k9–k5–k5) you used hr_crop_size = 17

In [5]:
!rm checkpoint/SRCNN955/*

In [6]:
!python precompute_patches.py dataset/train train_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 175 --augment

[+] Dumped 1879850 patches → train_patches_955.lmdb


In [7]:
!python precompute_patches.py dataset/validation valid_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 175 --augment

[+] Dumped 200200 patches → valid_patches_955.lmdb


In [8]:
# train SRCNN-955 model
!rm -rf dataset/*.npy
!python train.py  --steps=300000                    \
                  --architecture="955"              \
                  --batch_size=512                  \
                  --save-best-only=1                \
                  --save-every=1000                 \
                  --save-log=1                      \
                  --ckpt-dir="checkpoint/SRCNN955"  \
                  --num-worker=4

→ Loading train patches from LMDB: train_patches_955.lmdb
→ Loading valid patches from LMDB: valid_patches_955.lmdb
Training start..
[07:20:33] Starting training for 300000 steps (from step 0)
[07:20:38] Progress step 100/300000 — loss 0.1674265 — PSNR 11.807
[07:20:42] Progress step 200/300000 — loss 0.0851593 — PSNR 18.623
[07:20:45] Progress step 300/300000 — loss 0.0577239 — PSNR 20.912
[07:20:49] Progress step 400/300000 — loss 0.0439933 — PSNR 22.076
[07:20:52] Progress step 500/300000 — loss 0.0357567 — PSNR 22.771
[07:20:55] Progress step 600/300000 — loss 0.0302754 — PSNR 23.220
[07:20:58] Progress step 700/300000 — loss 0.0263423 — PSNR 23.568
[07:21:02] Progress step 800/300000 — loss 0.0234086 — PSNR 23.804
[07:21:05] Progress step 900/300000 — loss 0.0211131 — PSNR 24.010
[07:21:08] Progress step 1000/300000 — loss 0.0192870 — PSNR 24.158
[07:21:27] Eval @ 1000: train_loss 0.0192870 — train_PSNR 24.158 — val_loss 0.0027541 — val_PSNR 26.231
[07:21:27] Saved best model at s

# **Test**

In [9]:
# Test on Set5
!python test.py --scale=2 --architecture=955 --ckpt-path="default"
!python test.py --scale=3 --architecture=955 --ckpt-path="default"
!python test.py --scale=4 --architecture=955 --ckpt-path="default"

16.2664306640625
16.26265106201172
16.26767578125


In [10]:
# Test on FLIR data
!python test-ir.py --scale=2 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=3 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=4 --architecture=955 --ckpt-path="default"

20.232242584228516
20.23294194539388
20.232686360677082


# **Demo**

In [11]:
!python demo.py --image-path="dataset/test1.png" \
                --architecture="955"             \
                --ckpt-path="default"            \
                --scale=4